In [ ]:
def to_base_string(s):
    base_s = ""
    bases = ['A', 'T', 'G', 'C', '_']
    for i in range(len(s)):
        val = s[i]
        base_s += bases[val]
    return base_s

def to_base_list(base_s):
    s = []
    for i in range(len(base_s)):
        b = base_s[i]
        if(b == 'A'):
            s.append(0)
        elif(b == 'T'):
            s.append(1)
        elif(b =='G'):
            s.append(2)
        elif(b =='C'):
            s.append(3)
        elif(b=='_'):
            s.append(4)
        else:
            raise Exception(f"Invalid base string, contains character {b}.")
    return s

def tbl(s):
    return to_base_list(s)

def tbs(l):
    return to_base_string(l)

In [ ]:
def print_find_alignment_many(score,lst,s2,i1,i2):
    print(f"i1:{i1}, i2:{i2}")
    for i in range(len(lst)):
        print(i2 * '+' + lst[i][0] * "_" + tbs(lst[i][1]))
    print(i1 * '+' + tbs(s2))
    print(f"score:{score}")
    print("==========")
    
def show_alignment(s,i):
    pan = 10
    if(i < 0):
        pan = pan + i
    print(pan * '+' + i * '_' + to_base_string(s) + f"    :{i}")

In [ ]:
def calc_overlap_many(lst, s2, i1, i2):
    total_overlap = 0
    for i in range(len(lst)):
        index = lst[i][0]
        s1 = lst[i][1]
        
        overlap = min(len(s1) - i1 + index, len(s2) - i2 - index)
        if(overlap > 0):
            total_overlap += overlap
    return total_overlap

def calc_score_basic_many(lst, s2, i1, i2):
      
    total_score = 0
    overlap = calc_overlap_many(lst, s2,i1,i2)
    
    while(len(s2) > i2 and get_furtherest_index(lst) - 1 > i1):

        for i in range(len(lst)):
            s1 = lst[i][1]
            index = lst[i][0]
            
            if(index <= i2 and len(s1) + index > i1):
                score = -1
                if(s1[i1-index] == s2[i2]):
                    score = 1
                total_score += score
        i1 += 1
        i2 += 1
    return total_score + overlap

In [ ]:
def get_furtherest_index(lst):
    longest_length = len(lst[0][1])
    longest_length_i = lst[0][0]
    for i in range(1,len(lst)):
        if(len(lst[i][1]) + lst[i][0] > longest_length + longest_length_i):
            longest_length = len(lst[i][1])
            longest_length_i = lst[i][0]
    return longest_length + longest_length_i

def find_alignment_many(lst,s2, fb = False):
       
    i1 = get_furtherest_index(lst) -1
    i2 = 0
    
    max_score = -100
    max_s2 = []
    max_i2 = 0
    
    print_i1 = 0
    print_i2 = 0
    
    count = 0
    while(i2 < len(s2)):               
        score = calc_score_basic_many(lst, s2, i1,i2)
        if(fb):                       
            print_find_alignment_many(score,lst,s2,i1,i2)
            
        if(score > max_score):
            
            print_i1 = i1
            print_i2 = i2
            
            max_score = score
            max_s2 = s2
            if(i1 > 0):
                max_i2 = i1
            else:
                max_i2 = -1*i2
        if(i1 > 0):
            i1 -= 1
        else:
            i2 += 1
   
    return max_score, max_s2, max_i2

def align_many(sequences,s2, fb = False):
    score, s2, i2 = find_alignment_many(sequences,s2, fb) 
    return s2, i2

In [ ]:
#test 1
# lst = [
#     [0,to_base_list("AATTCC")],
#     [1,to_base_list("ATTCCG")],
#     [2,to_base_list("TTCCGG")],
#     [3,to_base_list("TCCGGA")],
#     [4,to_base_list("CCGGAA")]
# ]
# s = tbl("AATTCCGGAA")
# find_alignment_many(lst, s)

#test2
# lst = [
#     [1,to_base_list("AAA")],
#     [1,to_base_list("A")]
# ]
# s = tbl("AA")
# find_alignment_many(lst, s)

#test3
lst = [
    [0,tbl("CTCTTGGGGGATAAGCCTGTTATC")],
    [0,tbl("ACTCTTGGGGATAAGCCTGTTATCA")],
    [1,tbl("CTCTTGGGGATAAGCCTGTTATC")],
    [0,tbl("CTCTTGGGGGATAAGCCTGTTATC")],
    [0,tbl("CTCTTGGGGGATAAGCCTGTTATC")],
    [2,tbl("TCTTGGGGATAAGCCTGTTATCC")],
    [2,tbl("CTTGGGGGATAAGCCTGTTATCCA")],
    [3,tbl("CTTGGGGATAAGCCTGTTATCC")],
    [3,tbl("CTTGGGGATAAGCCTGTTATCCC")],
    [4,tbl("CTTGGGGATAAGCCTGTTATCCC")],
]
s = tbl("TGGGGATAAGCCTGTTATCCC")
find_alignment_many(lst, s, True)

In [ ]:
path = "d:/programming/python/data/assemblydata.npy" #path to the label / unassembled sequences
import numpy as np

file = np.load(path, allow_pickle=True)
sequences_list = []
sequences = file[0]
reference = file[1]

for i in range(len(sequences)):
    sequences_list.append(to_base_list(sequences[i]))

In [ ]:
import collections
alignments = []
max_length = 0

f = 0
t = 100

s1 = sequences_list[f]
s2 = sequences_list[f+1]
b2, index = align_many([[0,s1]],s2)
alignments.append([0, s1])
alignments.append([index,b2])

prev_index = 0

for i in range(f+1, t):

    window = 10
    sequences = []
    for j in range(1, window):
        if(i-j >= 0):
            sequences.append(alignments[i-j-f])
    s2 = sequences_list[i]

    sequence, index = align_many(sequences, s2)
    
    #debug
#     if(abs(index - prev_index) > 5):
#         align_many(sequences, s2, True)
#         print("======")
#     prev_index = index
    
    alignments.append([index,sequences_list[i]])
    if(index + len(s2) > max_length):
        max_length = index + len(s2)
    
for i in range(len(alignments)):
    show_alignment(alignments[i][1], alignments[i][0])

In [ ]:
import collections
dna = []
for col in range(max_length):
    col_bases = []
    for j in range(len(alignments)):   
        i = alignments[j][0]
        s = alignments[j][1]
        if(col >= i and col < i + len(s)):
            col_bases.append(s[col-i])

    freq_base = collections.Counter(col_bases).most_common(1)
    if(freq_base != []):
        dna.append(freq_base[0][0])
print(to_base_string(dna))